# Importing the libraries

# New Section

In [ ]:
!git clone https://github.com/mandar196/Hate_Speech_Detection-NLP.git

Cloning into 'Hate_Speech_Detection-NLP'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 19 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
!gdown --id 1ti6QbZh_j9rh9fhKGZIoThM7FN9tF3Un #training data

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1ti6QbZh_j9rh9fhKGZIoThM7FN9tF3Un
To: /content/labeled_data.csv
100% 2.57M/2.57M [00:00<00:00, 123MB/s]


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#to data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

#NLP tools
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

#train split and fit models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

#model selection
from sklearn.metrics import confusion_matrix, accuracy_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Importing the dataset

In [ ]:
dataset = pd.read_csv('/content/labeled_data.csv')
dataset.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [ ]:
dataset['class'] = dataset['class'].replace({2: 1})

In [ ]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,24783.0,12681.192027,7299.553863,0.0,6372.5,12703.0,18995.5,25296.0
count,24783.0,3.243473,0.883060,3.0,3.0,3.0,3.0,9.0
hate_speech,24783.0,0.280515,0.631851,0.0,0.0,0.0,0.0,7.0
offensive_language,24783.0,2.413711,1.399459,0.0,2.0,3.0,3.0,9.0
neither,24783.0,0.549247,1.113299,0.0,0.0,0.0,0.0,9.0
class,24783.0,0.942299,0.233182,0.0,1.0,1.0,1.0,1.0


In [ ]:
dt_trasformed = dataset[['class', 'tweet']]
y = dt_trasformed.iloc[:, :-1].values

In [ ]:
xy = np.ravel(y,order='C')

### Encoding the Dependent Variable

In [ ]:
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
# y = np.array(ct.fit_transform(y))

In [ ]:
print(y)

[[2]
 [1]
 [1]
 ...
 [1]
 [1]
 [2]]


I separated this y in two variables that we will use to fit hate speech models and offensive speech models

In [ ]:
# y_df = pd.DataFrame(y)
# y_hate = np.array(y_df[0])
# y_offensive = np.array(y_df[1])

In [ ]:
print(y_hate)
print(y_offensive)

[0. 0. 0. ... 0. 0. 0.]
[0. 1. 1. ... 1. 1. 0.]


## Cleaning the texts

In [ ]:
corpus = []
for i in range(0, 24783):
  review = re.sub('[^a-zA-Z]', ' ', dt_trasformed['tweet'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus).toarray()

## Splitting the dataset into the Training set and Test set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, xy, test_size = 0.30, random_state = 0)

In [ ]:
# dfd = pd.DataFrame(y_test)
# dfd.to_csv('ML_DAV_ytest.csv')

In [ ]:
# # print(type(data))
# dfdd = pd.DataFrame(X_test)
# dfdd.to_csv('ML_DAV_Xtest.csv')

In [ ]:
# d = pd.read_csv('/content/ML_DAV_Xtest.csv')
# # d.head(5)
# d.columns.values[0] = 'id'
# d.drop('id', inplace=True, axis=1)
# X_t = d.to_numpy()
# X_t

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Finding the best models to predict hate speech

**Naive Bayes**

In [ ]:
classifier_np = GaussianNB()
classifier_np.fit(X_train, y_train)

GaussianNB()

**Decision Tree**

In [ ]:
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

**KNN**

In [ ]:
classifier_knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_knn.fit(X_train, y_train)

KNeighborsClassifier()

**Logistic Regression**

In [ ]:
classifier_lr = LogisticRegression(random_state = 0)
classifier_lr.fit(X_train, y_train)

LogisticRegression(random_state=0)

**Random Forest**

In [ ]:
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier_rf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

SVM Classifier

In [ ]:
classifier_svm = svm.SVC()
classifier_svm.fit(X_train, y_train)

SVC()

XGBoost Classifier

In [ ]:
classifier_xgb= XGBClassifier()
classifier_xgb.fit(X_train, y_train)

XGBClassifier()

# Making the Confusion Matrix for each model

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#Naive Bayes
y_pred_np = classifier_np.predict(X_test)
cm = confusion_matrix(y_test, y_pred_np)
print(classification_report(y_test,y_pred_np))
print(cm)


              precision    recall  f1-score   support

           0       0.07      0.61      0.12       427
           1       0.95      0.47      0.63      7008

    accuracy                           0.48      7435
   macro avg       0.51      0.54      0.37      7435
weighted avg       0.90      0.48      0.60      7435

[[ 259  168]
 [3719 3289]]


In [ ]:
# y_pred_np = classifier_np.predict(X_t)
# cm = confusion_matrix(y_test, y_pred_np)
# print(classification_report(y_test,y_pred_np))
# print(cm)

In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_test)
cm = confusion_matrix(y_test, y_pred_knn)
print(classification_report(y_test,y_pred_knn))
print(cm)

              precision    recall  f1-score   support

           0       0.42      0.24      0.31       427
           1       0.95      0.98      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.69      0.61      0.64      7435
weighted avg       0.92      0.94      0.93      7435

[[ 103  324]
 [ 140 6868]]


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_test)
cm = confusion_matrix(y_test, y_pred_xgb)
print(classification_report(y_test,y_pred_xgb))
print(cm)

              precision    recall  f1-score   support

           0       0.65      0.09      0.16       427
           1       0.95      1.00      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.80      0.55      0.57      7435
weighted avg       0.93      0.94      0.93      7435

[[  40  387]
 [  22 6986]]


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_test)
cm = confusion_matrix(y_test, y_pred_svm)
print(classification_report(y_test,y_pred_svm))
print(cm)

              precision    recall  f1-score   support

           0       0.56      0.13      0.21       427
           1       0.95      0.99      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.75      0.56      0.59      7435
weighted avg       0.93      0.94      0.93      7435

[[  55  372]
 [  44 6964]]


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_test)
cm = confusion_matrix(y_test, y_pred_lr)
print(classification_report(y_test,y_pred_lr))
print(cm)

              precision    recall  f1-score   support

           0       0.45      0.19      0.26       427
           1       0.95      0.99      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.70      0.59      0.62      7435
weighted avg       0.92      0.94      0.93      7435

[[  80  347]
 [  97 6911]]


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_test)
cm = confusion_matrix(y_test, y_pred_dt)
print(classification_report(y_test,y_pred_dt))
print(cm)


              precision    recall  f1-score   support

           0       0.29      0.34      0.32       427
           1       0.96      0.95      0.95      7008

    accuracy                           0.91      7435
   macro avg       0.63      0.65      0.64      7435
weighted avg       0.92      0.91      0.92      7435

[[ 147  280]
 [ 355 6653]]


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred_rf)
print(classification_report(y_test,y_pred_rf))
print(cm)

              precision    recall  f1-score   support

           0       0.41      0.28      0.34       427
           1       0.96      0.98      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.68      0.63      0.65      7435
weighted avg       0.93      0.94      0.93      7435

[[ 121  306]
 [ 174 6834]]


In [ ]:
rf_score = accuracy_score(y_test, y_pred_rf)
knn_score = accuracy_score(y_test, y_pred_knn)
svm_score = accuracy_score(y_test, y_pred_svm)
xgb_score = accuracy_score(y_test, y_pred_xgb)
lr_score = accuracy_score(y_test, y_pred_lr)
dt_score = accuracy_score(y_test, y_pred_dt)
np_score = accuracy_score(y_test, y_pred_np)

print('Random Forest Accuracy: ', str(rf_score))
print('K Nearest Neighbours Accuracy: ', str(knn_score))
print('Support Vector Machine Accuracy: ', str(svm_score))
print('XGBoost Classifier Accuracy: ', str(xgb_score))
print('Logistic Regression Accuracy: ',str(lr_score))
print('Decision Tree Accuracy: ', str(dt_score))
print('Naive Bayes Accuracy: ', str(np_score))

Random Forest Accuracy:  0.9354404841963685
K Nearest Neighbours Accuracy:  0.9375924680564895
Support Vector Machine Accuracy:  0.9440484196368527
XGBoost Classifier Accuracy:  0.9449899125756557
Logistic Regression Accuracy:  0.9402824478816408
Decision Tree Accuracy:  0.9145931405514459
Naive Bayes Accuracy:  0.4772024209818426


So XGBoost Classifier looks better to predict hate speech based on this dataset. It's important to emphasize Random Forest,KNN and Decision Tree had great results as well. This Dataset looks like a product of artificial intelligence to classify hate and offensive speech.

In [ ]:
#Now finally saving the model with XGB Classifier.
import joblib
joblib.dump(classifier_xgb, 'hatespeech.pkl')

['hatespeech.pkl']

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
score_rf=precision_recall_fscore_support(y_test,y_pred_rf,average='weighted')
print(score_rf)

(0.9257295918948889, 0.9354404841963685, 0.9298400900345859, None)


In [ ]:
score_knn=precision_recall_fscore_support(y_test,y_pred_knn,average='weighted')
print(score_knn)

(0.9244493584103454, 0.9375924680564895, 0.9294274060991631, None)


In [ ]:
score_svm=precision_recall_fscore_support(y_test,y_pred_svm,average='weighted')
print(score_svm)

(0.9266785096833814, 0.9440484196368527, 0.9272431543170429, None)


In [ ]:
score_xgb=precision_recall_fscore_support(y_test,y_pred_xgb,average='weighted')
print(score_xgb)

(0.9301469114740184, 0.9449899125756557, 0.9251576605306921, None)


In [ ]:
score_lr=precision_recall_fscore_support(y_test,y_pred_lr,average='weighted')
print(score_lr)

(0.9234629023197887, 0.9402824478816408, 0.9284469336234399, None)


In [ ]:
score_dt=precision_recall_fscore_support(y_test,y_pred_dt,average='weighted')
print(score_dt)

(0.9213192813486385, 0.9145931405514459, 0.9178109360038883, None)


In [ ]:
score_np=precision_recall_fscore_support(y_test,y_pred_np,average='weighted')
print(score_np)

(0.9005020892683637, 0.4772024209818426, 0.59922542800809, None)


# Testing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Test on Large Facebook

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB!_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB!_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='micro')
print(score_np)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='macro')
print(score_np)

              precision    recall  f1-score   support

           0       0.30      0.16      0.21       929
           1       0.62      0.78      0.69      1621

    accuracy                           0.55      2550
   macro avg       0.46      0.47      0.45      2550
weighted avg       0.50      0.55      0.52      2550

[[ 151  778]
 [ 357 1264]]
(0.501780529975955, 0.5549019607843138, 0.515279699781726, None)
(0.5549019607843138, 0.5549019607843138, 0.5549019607843138, None)
(0.4581225369600592, 0.47115297139468587, 0.45015237290811405, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='micro')
print(score_knn)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='macro')
print(score_knn)

              precision    recall  f1-score   support

           0       0.51      0.02      0.04       929
           1       0.64      0.99      0.78      1621

    accuracy                           0.64      2550
   macro avg       0.58      0.50      0.41      2550
weighted avg       0.59      0.64      0.51      2550

[[  19  910]
 [  18 1603]]
(0.592573494352947, 0.636078431372549, 0.5073186505623893, None)
(0.636078431372549, 0.636078431372549, 0.636078431372549, None)
(0.5756982609350297, 0.5046739211997537, 0.40742877576347636, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='micro')
print(score_xgb)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='macro')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       929
           1       0.64      1.00      0.78      1621

    accuracy                           0.64      2550
   macro avg       0.32      0.50      0.39      2550
weighted avg       0.40      0.64      0.49      2550

[[   0  929]
 [   1 1620]]
(0.40400618466295896, 0.6352941176470588, 0.4939145154464665, None)
(0.6352941176470588, 0.6352941176470588, 0.6352941176470588, None)
(0.31777167516673205, 0.499691548426897, 0.3884892086330936, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='micro')
print(score_svm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='macro')
print(score_svm)

              precision    recall  f1-score   support

           0       0.67      0.00      0.00       929
           1       0.64      1.00      0.78      1621

    accuracy                           0.64      2550
   macro avg       0.65      0.50      0.39      2550
weighted avg       0.65      0.64      0.50      2550

[[   2  927]
 [   1 1620]]
(0.6471992424767316, 0.636078431372549, 0.49571509679386827, None)
(0.636078431372549, 0.636078431372549, 0.636078431372549, None)
(0.6513545347467609, 0.5007679746916979, 0.39082154654716506, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='micro')
print(score_lr)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='macro')
print(score_lr)

              precision    recall  f1-score   support

           0       0.50      0.02      0.04       929
           1       0.64      0.99      0.78      1621

    accuracy                           0.64      2550
   macro avg       0.57      0.50      0.41      2550
weighted avg       0.59      0.64      0.51      2550

[[  19  910]
 [  19 1602]]
(0.5875586986386911, 0.635686274509804, 0.5071154961531353, None)
(0.635686274509804, 0.635686274509804, 0.635686274509804, None)
(0.5688694267515924, 0.5043654696266507, 0.40726030129026813, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='micro')
print(score_dt)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='macro')
print(score_dt)

              precision    recall  f1-score   support

           0       0.41      0.10      0.16       929
           1       0.64      0.92      0.75      1621

    accuracy                           0.62      2550
   macro avg       0.52      0.51      0.46      2550
weighted avg       0.56      0.62      0.54      2550

[[  91  838]
 [ 131 1490]]
(0.5561968669700628, 0.62, 0.5373089955138265, None)
(0.62, 0.62, 0.62, None)
(0.5249721370855391, 0.5085702389719432, 0.4563723970634275, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='micro')
print(score_rf)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='macro')
print(score_rf)

              precision    recall  f1-score   support

           0       0.44      0.04      0.07       929
           1       0.64      0.97      0.77      1621

    accuracy                           0.63      2550
   macro avg       0.54      0.51      0.42      2550
weighted avg       0.57      0.63      0.51      2550

[[  34  895]
 [  43 1578]]
(0.5664917374515148, 0.6321568627450981, 0.5146660982599094, None)
(0.6321568627450981, 0.6321568627450981, 0.6321568627450981, None)
(0.5398249142689094, 0.5050358288581847, 0.41923932710527256, None)


## Test on Facebook

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_FB_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.42      0.08      0.14        59
           1       0.61      0.92      0.73        90

    accuracy                           0.59       149
   macro avg       0.51      0.50      0.44       149
weighted avg       0.53      0.59      0.50       149

[[ 5 54]
 [ 7 83]]
(0.530932085762341, 0.5906040268456376, 0.49748212837918027, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.50      0.02      0.03        59
           1       0.61      0.99      0.75        90

    accuracy                           0.60       149
   macro avg       0.55      0.50      0.39       149
weighted avg       0.56      0.60      0.47       149

[[ 1 58]
 [ 1 89]]
(0.5636899054923983, 0.6040268456375839, 0.466640019720597, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.60      1.00      0.75        90

    accuracy                           0.60       149
   macro avg       0.30      0.50      0.38       149
weighted avg       0.36      0.60      0.45       149

[[ 0 59]
 [ 0 90]]
(0.3648484302508896, 0.6040268456375839, 0.45491561596136026, None)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.60      1.00      0.75        90

    accuracy                           0.60       149
   macro avg       0.30      0.50      0.38       149
weighted avg       0.36      0.60      0.45       149

[[ 0 59]
 [ 0 90]]
(0.3648484302508896, 0.6040268456375839, 0.45491561596136026, None)


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classificat

In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.45      0.08      0.14        59
           1       0.61      0.93      0.74        90

    accuracy                           0.60       149
   macro avg       0.53      0.51      0.44       149
weighted avg       0.55      0.60      0.50       149

[[ 5 54]
 [ 6 84]]
(0.5476563121733825, 0.5973154362416108, 0.5016400060554069, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.40      0.20      0.27        59
           1       0.61      0.80      0.69        90

    accuracy                           0.56       149
   macro avg       0.50      0.50      0.48       149
weighted avg       0.52      0.56      0.52       149

[[12 47]
 [18 72]]
(0.5238508826349332, 0.5637583892617449, 0.5229508841445705, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.31      0.08      0.13        59
           1       0.59      0.88      0.71        90

    accuracy                           0.56       149
   macro avg       0.45      0.48      0.42       149
weighted avg       0.48      0.56      0.48       149

[[ 5 54]
 [11 79]]
(0.4825244739365192, 0.5637583892617449, 0.48076162959841884, None)


## Test on stormfront

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_SF_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_SF_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.04      0.08      0.06       342
           1       0.88      0.77      0.82      2942

    accuracy                           0.70      3284
   macro avg       0.46      0.43      0.44      3284
weighted avg       0.79      0.70      0.74      3284

[[  29  313]
 [ 676 2266]]
(0.7914167592138794, 0.6988428745432399, 0.7411488120432773, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.19      0.01      0.02       342
           1       0.90      0.99      0.94      2942

    accuracy                           0.89      3284
   macro avg       0.54      0.50      0.48      3284
weighted avg       0.82      0.89      0.85      3284

[[   4  338]
 [  17 2925]]
(0.8228970320215572, 0.8919001218026796, 0.8469000317247793, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       342
           1       0.90      1.00      0.94      2942

    accuracy                           0.90      3284
   macro avg       0.45      0.50      0.47      3284
weighted avg       0.80      0.90      0.85      3284

[[   0  342]
 [   2 2940]]
(0.8025059732301314, 0.8952496954933008, 0.8463446671405177, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       1.00      0.00      0.01       342
           1       0.90      1.00      0.95      2942

    accuracy                           0.90      3284
   macro avg       0.95      0.50      0.48      3284
weighted avg       0.91      0.90      0.85      3284

[[   1  341]
 [   0 2942]]
(0.9069485776021827, 0.896163215590743, 0.8473915981892935, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.16      0.02      0.04       342
           1       0.90      0.99      0.94      2942

    accuracy                           0.89      3284
   macro avg       0.53      0.50      0.49      3284
weighted avg       0.82      0.89      0.85      3284

[[   8  334]
 [  43 2899]]
(0.8196437753081961, 0.8852009744214373, 0.8454040261927145, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.14      0.09      0.11       342
           1       0.90      0.93      0.92      2942

    accuracy                           0.85      3284
   macro avg       0.52      0.51      0.51      3284
weighted avg       0.82      0.85      0.83      3284

[[  32  310]
 [ 195 2747]]
(0.8196934268087137, 0.8462241169305724, 0.8321583059939074, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.15      0.04      0.06       342
           1       0.90      0.97      0.93      2942

    accuracy                           0.88      3284
   macro avg       0.52      0.51      0.50      3284
weighted avg       0.82      0.88      0.84      3284

[[  14  328]
 [  80 2862]]
(0.8192557420015657, 0.8757612667478685, 0.8429396443328516, None)


## Test on Reddit

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_RE_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_RE_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

         0.0       0.27      0.04      0.07      5813
         1.0       0.50      0.89      0.64      6188

    accuracy                           0.48     12001
   macro avg       0.38      0.47      0.36     12001
weighted avg       0.39      0.48      0.36     12001

[[ 247 5566]
 [ 682 5506]]
(0.3851994044823664, 0.47937671860678277, 0.36446273725228434, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

         0.0       0.63      0.02      0.03      5813
         1.0       0.52      0.99      0.68      6188

    accuracy                           0.52     12001
   macro avg       0.58      0.50      0.35     12001
weighted avg       0.57      0.52      0.36     12001

[[  88 5725]
 [  51 6137]]
(0.5734218666394497, 0.5187067744354638, 0.3649470698230344, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

         0.0       0.76      0.00      0.01      5813
         1.0       0.52      1.00      0.68      6188

    accuracy                           0.52     12001
   macro avg       0.64      0.50      0.34     12001
weighted avg       0.63      0.52      0.35     12001

[[  19 5794]
 [   6 6182]]
(0.6342904602082536, 0.5167069410882427, 0.3541312292989162, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      5813
         1.0       0.52      1.00      0.68      6188

    accuracy                           0.51     12001
   macro avg       0.26      0.50      0.34     12001
weighted avg       0.27      0.51      0.35     12001

[[   0 5813]
 [   9 6179]]
(0.26568035607884355, 0.5148737605199567, 0.3504993212428485, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

         0.0       0.66      0.07      0.12      5813
         1.0       0.52      0.97      0.68      6188

    accuracy                           0.53     12001
   macro avg       0.59      0.52      0.40     12001
weighted avg       0.59      0.53      0.41     12001

[[ 386 5427]
 [ 195 5993]]
(0.5923955065692593, 0.5315390384134655, 0.40947463427062014, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

         0.0       0.58      0.20      0.29      5813
         1.0       0.53      0.87      0.66      6188

    accuracy                           0.54     12001
   macro avg       0.56      0.53      0.48     12001
weighted avg       0.56      0.54      0.48     12001

[[1142 4671]
 [ 816 5372]]
(0.5583186924201691, 0.542788100991584, 0.4836782196549771, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

         0.0       0.60      0.09      0.15      5813
         1.0       0.52      0.94      0.67      6188

    accuracy                           0.53     12001
   macro avg       0.56      0.52      0.41     12001
weighted avg       0.56      0.53      0.42     12001

[[ 511 5302]
 [ 343 5845]]
(0.5602022711448125, 0.529622531455712, 0.4219662613476397, None)


## Test on Davidson

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_DAV_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_DAV_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.07      0.61      0.12       427
           1       0.95      0.47      0.63      7008

    accuracy                           0.48      7435
   macro avg       0.51      0.54      0.37      7435
weighted avg       0.90      0.48      0.60      7435

[[ 259  168]
 [3719 3289]]
(0.9005020892683637, 0.4772024209818426, 0.59922542800809, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.42      0.24      0.31       427
           1       0.95      0.98      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.69      0.61      0.64      7435
weighted avg       0.92      0.94      0.93      7435

[[ 103  324]
 [ 140 6868]]
(0.9244493584103454, 0.9375924680564895, 0.9294274060991631, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.65      0.09      0.16       427
           1       0.95      1.00      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.80      0.55      0.57      7435
weighted avg       0.93      0.94      0.93      7435

[[  40  387]
 [  22 6986]]
(0.9301469114740184, 0.9449899125756557, 0.9251576605306921, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.56      0.13      0.21       427
           1       0.95      0.99      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.75      0.56      0.59      7435
weighted avg       0.93      0.94      0.93      7435

[[  55  372]
 [  44 6964]]
(0.9266785096833814, 0.9440484196368527, 0.9272431543170429, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.45      0.19      0.26       427
           1       0.95      0.99      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.70      0.59      0.62      7435
weighted avg       0.92      0.94      0.93      7435

[[  80  347]
 [  97 6911]]
(0.9234629023197887, 0.9402824478816408, 0.9284469336234399, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.29      0.34      0.32       427
           1       0.96      0.95      0.95      7008

    accuracy                           0.91      7435
   macro avg       0.63      0.65      0.64      7435
weighted avg       0.92      0.91      0.92      7435

[[ 147  280]
 [ 355 6653]]
(0.9213192813486385, 0.9145931405514459, 0.9178109360038883, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.41      0.28      0.34       427
           1       0.96      0.98      0.97      7008

    accuracy                           0.94      7435
   macro avg       0.68      0.63      0.65      7435
weighted avg       0.93      0.94      0.93      7435

[[ 121  306]
 [ 174 6834]]
(0.9257295918948889, 0.9354404841963685, 0.9298400900345859, None)


## Test on Waseem

In [ ]:
d = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_WS_Xtest.csv')
# d.head(5)
d.columns.values[0] = 'id'
d.drop('id', inplace=True, axis=1)
X_t = d.to_numpy()
de = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/ML_TEST/Data/ML_WS_ytest.csv')
# d.head(5)
de.columns.values[0] = 'id'
de.drop('id', inplace=True, axis=1)
y_t = de.to_numpy()
# X_t

In [ ]:
#Naive Bayes by Davidson or itself
y_pred_np = classifier_np.predict(X_t)
cm = confusion_matrix(y_t, y_pred_np)
print(classification_report(y_t,y_pred_np))
print(cm)
score_np=precision_recall_fscore_support(y_t,y_pred_np,average='weighted')
print(score_np)

              precision    recall  f1-score   support

           0       0.17      0.08      0.11       898
           1       0.72      0.85      0.78      2500

    accuracy                           0.65      3398
   macro avg       0.44      0.47      0.45      3398
weighted avg       0.57      0.65      0.60      3398

[[  75  823]
 [ 375 2125]]
(0.5743778299364114, 0.6474396703943496, 0.6033498502237933, None)


In [ ]:
#KNN
y_pred_knn = classifier_knn.predict(X_t)
cm = confusion_matrix(y_t, y_pred_knn)
print(classification_report(y_t,y_pred_knn))
print(cm)
score_knn=precision_recall_fscore_support(y_t,y_pred_knn,average='weighted')
print(score_knn)

              precision    recall  f1-score   support

           0       0.29      0.01      0.02       898
           1       0.74      0.99      0.84      2500

    accuracy                           0.73      3398
   macro avg       0.51      0.50      0.43      3398
weighted avg       0.62      0.73      0.63      3398

[[   9  889]
 [  22 2478]]
(0.6181949727298984, 0.7319011183048852, 0.6266071756967629, None)


In [ ]:
#XGBoost Classifier
y_pred_xgb = classifier_xgb.predict(X_t)
cm = confusion_matrix(y_t, y_pred_xgb)
print(classification_report(y_t,y_pred_xgb))
print(cm)
score_xgb=precision_recall_fscore_support(y_t,y_pred_xgb,average='weighted')
print(score_xgb)

              precision    recall  f1-score   support

           0       0.67      0.00      0.00       898
           1       0.74      1.00      0.85      2500

    accuracy                           0.74      3398
   macro avg       0.70      0.50      0.43      3398
weighted avg       0.72      0.74      0.62      3398

[[   2  896]
 [   1 2499]]
(0.7177377434472269, 0.7360211889346675, 0.6249498401362875, None)


In [ ]:
#SVM
y_pred_svm = classifier_svm.predict(X_t)
cm = confusion_matrix(y_t, y_pred_svm)
print(classification_report(y_t,y_pred_svm))
print(cm)
score_svm=precision_recall_fscore_support(y_t,y_pred_svm,average='weighted')
print(score_svm)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       898
           1       0.74      1.00      0.85      2500

    accuracy                           0.74      3398
   macro avg       0.37      0.50      0.42      3398
weighted avg       0.54      0.74      0.62      3398

[[   0  898]
 [   1 2499]]
(0.5412368320695666, 0.7354326074161271, 0.6235650393557124, None)


In [ ]:
#Logistic Regression
y_pred_lr=classifier_lr.predict(X_t)
cm = confusion_matrix(y_t, y_pred_lr)
print(classification_report(y_t,y_pred_lr))
print(cm)
score_lr=precision_recall_fscore_support(y_t,y_pred_lr,average='weighted')
print(score_lr)

              precision    recall  f1-score   support

           0       0.52      0.02      0.03       898
           1       0.74      0.99      0.85      2500

    accuracy                           0.74      3398
   macro avg       0.63      0.51      0.44      3398
weighted avg       0.68      0.74      0.63      3398

[[  16  882]
 [  15 2485]]
(0.6793991218605318, 0.7360211889346675, 0.6323453749725536, None)


In [ ]:
#Decision Tree
y_pred_dt = classifier_dt.predict(X_t)
cm = confusion_matrix(y_t, y_pred_dt)
print(classification_report(y_t,y_pred_dt))
print(cm)
score_dt=precision_recall_fscore_support(y_t,y_pred_dt,average='weighted')
print(score_dt)

              precision    recall  f1-score   support

           0       0.23      0.08      0.12       898
           1       0.73      0.91      0.81      2500

    accuracy                           0.69      3398
   macro avg       0.48      0.49      0.46      3398
weighted avg       0.60      0.69      0.63      3398

[[  70  828]
 [ 231 2269]]
(0.6004851224199482, 0.6883460859329017, 0.6273786885366395, None)


In [ ]:
#Random Florest
y_pred_rf = classifier_rf.predict(X_t)
cm = confusion_matrix(y_t, y_pred_rf)
print(classification_report(y_t,y_pred_rf))
print(cm)
score_rf=precision_recall_fscore_support(y_t,y_pred_rf,average='weighted')
print(score_rf)

              precision    recall  f1-score   support

           0       0.22      0.02      0.03       898
           1       0.73      0.98      0.84      2500

    accuracy                           0.72      3398
   macro avg       0.48      0.50      0.44      3398
weighted avg       0.60      0.72      0.63      3398

[[  17  881]
 [  60 2440]]
(0.5988981418478444, 0.7230723955267805, 0.6260078545307758, None)
